In [1]:
import pandas as pd
import pathlib as pl
import subprocess

In [3]:
def in_notebook():
    try:
        from IPython import get_ipython
        if 'IPKernelApp' not in get_ipython().config:  # pragma: no cover
            return False
    except ImportError:
        return False
    except AttributeError:
        # In a terminal/shell, get_ipython() will return None
        return False
    return True

In [4]:
here = pl.Path.cwd() if in_notebook() else pl.Path(__file__).parent

manuscript_version_path = here / 'MANUSCRIPT_VERSION'
with manuscript_version_path.open('r') as file:
    version = file.read().strip()
    
alias_table = pd.read_csv(here / 'figure_aliases.csv')
alias_to_number = alias_table.set_index('alias')[version].to_dict()
number_to_alias = alias_table.set_index(version)['alias'].to_dict()

parent_directories = list(here.glob('fig_*'))

In [9]:
def rename_directory(d, dry_run=dry_run):
    dir_parts = d.name.split('__')
    alias = dir_parts[1]
    number = alias_to_number[alias]
    new_name = f'fig_{number}__{alias}'
    git_mv_cmd = ['git', 'mv', str(d), str(d.with_name(new_name))]
    if new_name == d.name:
        print(f"Directory {d.name} already has the correct name.")
        return new_name
    print(' '.join(git_mv_cmd))
    if not dry_run:
        subprocess.run(git_mv_cmd)
        
    return new_name
        
def rename_file(f, dry_run=dry_run):
    fig_parts = f.stem.split('__')
    assert len(fig_parts) <= 3, f"Unexpected file naming format: {f.name}"
    alias = fig_parts[1]
    number = alias_to_number[alias]
    if len(fig_parts) == 3:
        extra = '__' + fig_parts[2]
    else:
        extra = ''
    new_name = f'fig_{number}__{alias}{extra}'
    git_mv_cmd = ['git', 'mv', str(f), str(f.with_stem(new_name))]
    if new_name == f.stem:
        print(f"File {f.name} already has the correct name.")
        return new_name
    print(' '.join(git_mv_cmd))
    if not dry_run:
        subprocess.run(git_mv_cmd)
    return new_name
    
for dry_run in [True, False]:
    # Do a dry run first to ensure there are no errors
    print(f"{'dry run' if dry_run else 'actual run'} mode...\n")
    for d in parent_directories:
        assert d.is_dir(), f"{d} is not a directory"
        new_dir_name = rename_directory(d, dry_run=dry_run)
        
        if not dry_run:
            d = d.with_name(new_dir_name)

        for ext in ['.png', '.svg', '.ipynb']:
            for f in d.rglob(f'*{ext}'):
                if not f.is_file():
                    continue
                rename_file(f, dry_run=dry_run)
        print()

dry run mode...

Directory fig_S3__quick_nematode already has the correct name.
File fig_S3A__sturdy_orca.png already has the correct name.
File fig_S3B__covert_parrot.png already has the correct name.
File fig_S3B__covert_parrot.svg already has the correct name.
File fig_S3A__sturdy_orca.svg already has the correct name.
File fig_S3__quick_nematode__tripartite_competition.ipynb already has the correct name.

Directory fig_ED1__shy_quokka already has the correct name.
git mv /home/john/projects/can-manuscript/fig_ED1__shy_quokka/graphics/fig_ED1Eii__big_zebra.png /home/john/projects/can-manuscript/fig_ED1__shy_quokka/graphics/fig_ED1Fii__big_zebra.png
git mv /home/john/projects/can-manuscript/fig_ED1__shy_quokka/graphics/fig_ED1D__strong_xerus.png /home/john/projects/can-manuscript/fig_ED1__shy_quokka/graphics/fig_ED1E__strong_xerus.png
git mv /home/john/projects/can-manuscript/fig_ED1__shy_quokka/graphics/fig_ED1Ci__fast_vole.png /home/john/projects/can-manuscript/fig_ED1__shy_quokka/